In [1]:
import numpy as np
import scipy

# Problem 2 (QR decomposition)
## Part (a)

In [2]:
def projection(projecting_vector, projected_onto, inner_product=np.dot):
    numerator = inner_product(projecting_vector, projected_onto)
    denominator = inner_product(projected_onto, projected_onto)
    return (numerator / denominator) * projected_onto  

In [3]:
def gram_schmidt(vectors, inner_product=np.dot):
    n, m = vectors.shape
    out_vectors = np.zeros((n, m))
    
    for i in range(m):
        curr_og_vector = vectors[:, i]
        new_vector = curr_og_vector.copy()
        for j in range(0, i):
            new_vector -= projection(curr_og_vector, out_vectors[:, j])
        length = np.sqrt(inner_product(new_vector, new_vector))
        out_vectors[:, i] = new_vector / length
    
    return out_vectors

In [4]:
def qr_decomposition(input_matrix: np.array, gram_schmidt_method=gram_schmidt):
    gram_schmidt_cols = gram_schmidt_method(input_matrix.copy())
    
    Q = gram_schmidt_cols.copy()
    R = gram_schmidt_cols.T @ input_matrix
    
    R_rows, R_cols = R.shape
    
    R = np.array([[R[i, j] if j >= i else 0 for j in range(R_cols)] for i in range(R_rows)])
    
    return Q, R

#### Test it out!

In [5]:
input_matrix = np.random.rand(3, 3)
input_matrix = np.array(input_matrix).astype(float)

In [6]:
print(f"Your input matrix looks like: \n{input_matrix}")

Your input matrix looks like: 
[[0.3465309  0.39045702 0.7074207 ]
 [0.63288504 0.75908185 0.29824815]
 [0.2285009  0.46699531 0.74452241]]


In [7]:
Q, R = qr_decomposition(input_matrix, gram_schmidt_method=gram_schmidt)
print(f"The output Q @ R looks like: \n{Q @ R}")

The output Q @ R looks like: 
[[0.3465309  0.39045702 0.7074207 ]
 [0.63288504 0.75908185 0.29824815]
 [0.2285009  0.46699531 0.74452241]]


## Part (b)

In [8]:
def modified_gram_schmidt(vectors, inner_product=np.dot):
    n, m = vectors.shape
    out_vectors = vectors.copy()
    
    for j in range(m):
        out_vectors[:, j] = out_vectors[:, j] / np.sqrt(inner_product(out_vectors[:, j], out_vectors[:, j]))
        for i in range(j+1, m):
            out_vectors[:, i] = out_vectors[:, i] - (inner_product(out_vectors[:, i], out_vectors[:, j]) * out_vectors[:, j])
    
    return out_vectors

#### Test it out!

In [9]:
new_input_matrix = np.random.rand(3, 3)
new_input_matrix = np.array(input_matrix).astype(float)

In [10]:
print(f"Your input matrix looks like: \n{new_input_matrix}")

Your input matrix looks like: 
[[0.3465309  0.39045702 0.7074207 ]
 [0.63288504 0.75908185 0.29824815]
 [0.2285009  0.46699531 0.74452241]]


In [11]:
new_Q, new_R = qr_decomposition(input_matrix, gram_schmidt_method=modified_gram_schmidt)
print(f"The output Q @ R looks like: \n{new_Q @ new_R}")

The output Q @ R looks like: 
[[0.3465309  0.39045702 0.7074207 ]
 [0.63288504 0.75908185 0.29824815]
 [0.2285009  0.46699531 0.74452241]]


## Part (c, d)

In [12]:
def check_stability_of_hilbert(size_of_hilbert_matrix):
    print(f"Size of Hilbert matrix = {size_of_hilbert_matrix}")
    print("-"*40)
    hilbert_matrix = scipy.linalg.hilbert(size_of_hilbert_matrix)
    
    Q, R = qr_decomposition(hilbert_matrix, gram_schmidt_method=gram_schmidt)
    modified_Q, modified_R = qr_decomposition(hilbert_matrix, gram_schmidt_method=modified_gram_schmidt)
    
    out1 = np.linalg.norm(hilbert_matrix - (Q @ R))
    print(f"||H - QR|| for classical GS: {out1}")
    out2 = np.linalg.norm(hilbert_matrix - (modified_Q @ modified_R))
    print(f"||H - QR|| for modified GS: {out2}")
    
    out3 = np.linalg.norm(np.eye(size_of_hilbert_matrix) - (Q.T @ Q))
    print(f"||I - Q'Q|| for classical GS: {out3}")
    out4 = np.linalg.norm(np.eye(size_of_hilbert_matrix) - (modified_Q.T @ modified_Q))
    print(f"||I - Q'Q|| for modified GS: {out4}")

In [13]:
check_stability_of_hilbert(size_of_hilbert_matrix=10)

Size of Hilbert matrix = 10
----------------------------------------
||H - QR|| for classical GS: 4.928357553657667e-06
||H - QR|| for modified GS: 3.941156461239072e-06
||I - Q'Q|| for classical GS: 3.4657738578377857
||I - Q'Q|| for modified GS: 5.198983345447229e-05


In [14]:
check_stability_of_hilbert(size_of_hilbert_matrix=100)

Size of Hilbert matrix = 100
----------------------------------------
||H - QR|| for classical GS: 0.2049720681307243
||H - QR|| for modified GS: 1.0880299442129764
||I - Q'Q|| for classical GS: 92.41538065080829
||I - Q'Q|| for modified GS: 3.7148202930777665


## Part (e)

Based on the results above, we can see that the QR decomposition using modified Gram-Schmidt exhibits more stability than the QR decomposition using classical Gram-Schmidt, especially for large, difficult/ill-conditioned matrices. The value of $||I - Q'Q||$ is much smaller when using the modified method, and the value of $||A - QR||$ is almost always smaller when using the modified method (there is a small difference on the $100 \times 100$ Hilbert matrix, but this observation was confirmed on large, random matrices $A$ generated by NumPy).